# Homework 2

## Imports

In [1]:
import nbtools
nbtools.setup_nb()

In [15]:
import itertools
import sympy
from sympy.diffgeom import Manifold, Patch
from pystein import coords, metric, curvature
from pystein.utilities import tensor_pow as tpow, full_simplify

## Problem 2

In [3]:
M = Manifold('M', dim=3)
P = Patch('origin', M)
r, theta, phi = sympy.symbols('r theta phi', nonnegative=True)
cs = coords.CoordSystem('spherical-polar', P, [r, theta, phi])
dr, dtheta, dphi = cs.base_oneforms()


ds2 = tpow(dr, 2) + r ** 2 * tpow(dtheta, 2) + r ** 2 * sympy.sin(theta) ** 2 * tpow(dphi, 2)
gamma = metric.Metric(twoform=ds2)

gamma

r**2*sin(theta)**2*TensorProduct(dphi, dphi) + r**2*TensorProduct(dtheta, dtheta) + TensorProduct(dr, dr)

In [16]:
christoffels = [((i, j, k), full_simplify(curvature.christoffel_symbol_component(i, j, k, gamma))) 
                for i, j, k in itertools.product(range(3), range(3), range(3))]

In [49]:
curvature.display_components([c for c in christoffels if c[1]])

In [19]:
riccis = [((i, j), full_simplify(curvature.ricci_tensor_component(i, j, gamma).doit())) 
          for i, j in itertools.product(range(3), range(3))]

In [20]:
curvature.display_components(riccis)

 Computing $\beta_j$

In [22]:
M = sympy.symbols('M')

In [23]:
beta_j = sympy.Matrix([sympy.sqrt(2 * M / r), 0, 0])
beta_j

Matrix([
[sqrt(2)*sqrt(M/r)],
[                0],
[                0]])

In [25]:
(gamma.matrix * beta_j).T

Matrix([[sqrt(2)*sqrt(M/r), 0, 0]])

In [80]:
def cov_deriv_one_form(i, j, form, g):
    coord_syms = g.coord_system.base_symbols()
    v = sympy.Derivative(form[j], coord_syms[i])
    
    for k in range(len(coord_syms)):
        v -= curvature.christoffel_symbol_component(k, i, j, g) * form[k]

    return full_simplify(v.doit())

In [81]:
def k_ij(i, j, beta_j, g, alpha = 1):
    d_i_beta_j = cov_deriv_one_form(i, j, beta_j, g)
    d_j_beta_i = cov_deriv_one_form(j, i, beta_j, g)
    return - sympy.Rational(1, 2) / alpha * (d_i_beta_j + d_j_beta_i)

In [83]:
K_ij = sympy.Matrix([[k_ij(0, j, beta_j, gamma) for j in range(3)],
                     [k_ij(1, j, beta_j, gamma) for j in range(3)],
                     [k_ij(2, j, beta_j, gamma) for j in range(3)]])

In [85]:
K_ij = - (d_i_beta_j + d_j_beta_i) / 2
K_ij #/ beta_j[0]

Matrix([
[sqrt(2)*sqrt(M/r)/(2*r),                    0,                                  0],
[                      0, -sqrt(2)*r*sqrt(M/r),                                  0],
[                      0,                    0, -sqrt(2)*r*sqrt(M/r)*sin(theta)**2]])

$$K_{ij} \approx \frac{1}{r} \beta^r \gamma_{ij}$$

In [86]:
sympy.trace(gamma.matrix.inv() * K_ij)

-3*sqrt(2)*sqrt(M/r)/(2*r)